# Lab 1: Implementing self-editing memory from scratch


<div style="background-color:grey; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
There are a variety of ways to enable long-term memory in LLM agents, such as RAG and recursive summarization. The MemGPT paper first introduced the notion of *self-editing memory*. Essentially, offload memory management to the LLM. After all, the LLM is the most "intelligent" part of our programs, so why not have the LLM figure out memory instead of hard coding some solution?

In this section, we'll walk through how to use OpenAI's tool calling to implement some simple memory management tools.
</div>

In [1]:
# Install the Google Generative AI library
#!pip install -q google-generativeai

import google.generativeai as genai
import os
import sys
import uuid
import json
import time
from IPython.display import display, Markdown
# Import the function from your helper file
from helper import get_gemini_api_key

def print_markdown(text):
    """Prints text as markdown in a notebook."""
    display(Markdown(text))

# --- Configuration ---
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key is missing. Please ensure your helper.py file returns a valid key.")
    genai.configure(api_key=api_key)
    print("\nGemini API configured successfully!")
except Exception as e:
    print(f"An error occurred during configuration: {e}")



Gemini API configured successfully!


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Core Agent and Tool Logic
This section contains the reusable GeminiAgent class, the tool definitions and implementations, and the main run_agent_step function that handles the interaction loop.

In [ ]:
# --- Agent Class ---
class GeminiAgent:
    """A class to simulate an agent's state and memory."""
    def __init__(self, name, model_name, system_prompt="", memory_blocks=None, tools=None):
        self.id = f"agent-{uuid.uuid4()}"
        self.name = name
        self.memory_blocks = memory_blocks if memory_blocks else {}
        self.tools = tools if tools else []
        self.system_prompt = system_prompt
        self.model = genai.GenerativeModel(
            model_name=model_name,
            tools=self.tools
        )

    def get_formatted_memory(self):
        """Formats memory blocks into a string for the system prompt."""
        if not self.memory_blocks:
            return ""
        formatted_string = "--- CORE MEMORY ---\n"
        for label, value in self.memory_blocks.items():
            val_str = json.dumps(value) if isinstance(value, list) else str(value)
            formatted_string += f"<{label}>\n{val_str}\n</{label}>\n"
        return formatted_string.strip()

# --- Reusable Printing Function ---
def print_message(message_type, content):
    """Prints formatted messages based on their type."""
    if message_type == "reasoning":
        print(f"🧠 Reasoning: {content}")
    elif message_type == "assistant":
        print(f"🤖 Agent: {content}")
    elif message_type == "tool_call":
        tool_name = content.get("name", "N/A")
        arguments = content.get("arguments", {})
        print(f"🔧 Tool Call: {tool_name}\n{json.dumps(arguments, indent=2)}")
    elif message_type == "tool_return":
        print(f"🔧 Tool Return: {content}")
    elif message_type == "user":
        print(f"👤 User Message: {content}")
    else:
        print(content)
    print("-----------------------------------------------------")

# --- Tool Definitions (for the model) ---
def core_memory_save(section: str, memory: str):
    """
    Save important information about you, the agent, or the human you are chatting with.
    Args:
        section (str): Must be either 'human' (to save information about the human) or 'agent' (to save information about yourself).
        memory (str): Memory to save in the section.
    """
    pass

# --- Tool Implementations (for Python) ---
def _core_memory_save_impl(agent: "GeminiAgent", section: str, memory: str):
    """Implementation for saving to core memory."""
    if section in agent.memory_blocks:
        agent.memory_blocks[section] += f"\n{memory}"
    else:
        agent.memory_blocks[section] = memory
    return f"Memory saved to section '{section}'. Current memory: {json.dumps(agent.memory_blocks)}"

# --- Generic Agent Interaction Loop ---
def run_agent_step(agent, tool_registry, user_message, chat_history=None):
    """Handles a multi-step agent interaction."""
    if chat_history is None:
        chat_history = []

    print_message("user", user_message)
    
    system_prompt = (
        f"{agent.system_prompt}\n"
        "You must either call a tool (core_memory_save) or write a response to the user. "
        "Do not take the same actions multiple times! "
        "When you learn new information, make sure to always call the core_memory_save tool."
    )
    
    full_prompt = f"{system_prompt}\n\n{agent.get_formatted_memory()}\n\n**Task:**\n{user_message}"
    
    messages = chat_history + [{'role': 'user', 'parts': [{'text': full_prompt}]}]

    while True:
        response = agent.model.generate_content(messages, tools=agent.tools)
        time.sleep(1)
        
        if not response.candidates:
            final_text = "No response was generated. This might be due to safety settings."
            break
            
        message = response.candidates[0].content
        messages.append(message)

        if not any(part.function_call for part in message.parts):
            final_text = message.parts[0].text if message.parts else "Task complete."
            break

        tool_response_parts = []
        for part in message.parts:
            if not part.function_call: continue
            fc = part.function_call
            tool_name = fc.name
            tool_args = dict(fc.args)
            print_message("reasoning", f"Model wants to call `{tool_name}`.")
            print_message("tool_call", {"name": tool_name, "arguments": tool_args})
            
            # Pass the agent instance to the implementation
            result = tool_registry[tool_name](agent=agent, **tool_args)
            
            print_message("tool_return", result)
            tool_response_parts.append({"function_response": {"name": tool_name, "response": {"content": result}}})
        
        messages.append({"role": "tool", "parts": tool_response_parts})

    print_message("assistant", final_text)
    return final_text, messages


### 3. Section 1: Implementing Editable Memory
This section demonstrates the core concept of self-editing memory by creating an agent that can save information to its own memory blocks.

In [3]:
# --- 1. A simple agent's context window ---
model = "gemini-1.5-flash"
system_prompt = "You are a chatbot."
chat_completion = genai.GenerativeModel(model).generate_content(
    [
        {'role': 'user', 'parts': [{'text': system_prompt}]},
        {'role': 'user', 'parts': [{'text': "What is my name?"}]}
    ]
)
print(f"🤖 Agent: {chat_completion.text}")
print("="*50 + "\n")

# --- 2. Adding memory to the context ---
agent_memory = {"human": "Name: Bob"}
system_prompt_with_memory = (
    "You are a chatbot. "
    "You have a section of your context called [MEMORY] "
    "that contains information relevant to your conversation."
)
full_prompt = f"{system_prompt_with_memory}\n\n[MEMORY]\n{json.dumps(agent_memory)}"
chat_completion = genai.GenerativeModel(model).generate_content(
    [
        {'role': 'user', 'parts': [{'text': full_prompt}]},
        {'role': 'user', 'parts': [{'text': "What is my name?"}]}
    ]
)
print(f"🤖 Agent: {chat_completion.text}")
print("="*50 + "\n")

# --- 3. Modifying memory with tools ---
editable_memory_agent = GeminiAgent(
    name="editable_memory_agent",
    model_name="gemini-1.5-flash",
    system_prompt=system_prompt_with_memory,
    memory_blocks={"human": "", "agent": ""},
    tools=[core_memory_save]
)

tool_registry = {
    "core_memory_save": _core_memory_save_impl
}

# --- 4. Implementing an agentic loop ---
final_response, chat_history = run_agent_step(
    agent=editable_memory_agent,
    tool_registry=tool_registry,
    user_message="my name is bob."
)

# --- 5. Verifying the memory update ---
print("\nUpdated Agent Memory:")
print(json.dumps(editable_memory_agent.memory_blocks, indent=2))

# --- 6. Running the next agent step ---
final_response, chat_history = run_agent_step(
    agent=editable_memory_agent,
    tool_registry=tool_registry,
    user_message="what is my name",
    chat_history=chat_history
)


🤖 Agent: I do not know your name.  I have no memory of past conversations and no access to personal information about you unless you explicitly provide it.


🤖 Agent: Your name is Bob.


👤 User Message: my name is bob.
-----------------------------------------------------
🧠 Reasoning: Model wants to call `core_memory_save`.
-----------------------------------------------------
🔧 Tool Call: core_memory_save
{
  "memory": "My name is bob.",
  "section": "human"
}
-----------------------------------------------------
🔧 Tool Return: Memory saved to section 'human'. Current memory: {"human": "\nMy name is bob.", "agent": ""}
-----------------------------------------------------
🤖 Agent: OK. I'll remember that your name is Bob.

-----------------------------------------------------

Updated Agent Memory:
{
  "human": "\nMy name is bob.",
  "agent": ""
}
👤 User Message: what is my name
-----------------------------------------------------
🤖 Agent: Your name is Bob.

--------------------------